## 13.2 ローソク足チャートと連結散布図

In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime

import plotly
from plotly import graph_objects as go
from plotly import express as px
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import Template
from plotly import callbacks

# サンプルデータの日付
date_list = [
    datetime(year=2099, month=4, day=1),
    datetime(year=2099, month=4, day=2),
]

# サンプルデータの始値と終値
open_list = [20, 35]
close_list = [30, 25]

# サンプルデータの高値と安値
high_list = [40, 45]
low_list = [10, 15]

# Traceを作成
trace = go.Candlestick(
    x=date_list,        # x軸に使用する変数
    open=open_list,     # 始値に使用する変数
    close=close_list,   # 終値に使用する変数
    high=high_list,     # 高値に使用する変数
    low=low_list        # 安値に使用する変数
)   # サンプルデータのローソク足チャート

# 独自テンプレートを作成
with open('custom_white.json') as f:
    custom_white_dict = json.load(f)
    template = Template(custom_white_dict)

# Layoutを作成
layout = go.Layout(
    template=template,
    title='Candlestick sample',
    xaxis={'title': 'Date'},
    yaxis={'title': 'Stock'}
)

# Figureを作成
figure = go.Figure(trace, layout)

figure

In [2]:
# Hello-World-StockデータセットからDataFrameを読み込み
df = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/datasets/master/hello-world-stock.csv',
    usecols=['Date', 'Open', 'High', 'Low', 'Close', 'Stock'],
    parse_dates=[0]
)

df

,Date,Open,High,Low,Close,Stock
0,2017-12-29,170.52,170.5900,169.2200,169.23,AAPL
1,2017-12-28,171.00,171.8500,170.4800,171.08,AAPL
2,2017-12-27,170.10,170.7800,169.7100,170.60,AAPL
3,2017-12-26,170.80,171.4700,169.6790,170.57,AAPL
4,2017-12-22,174.68,175.4240,174.5000,175.01,AAPL
...,...,...,...,...,...,...
2256,2015-01-08,89.38,90.2000,88.1500,90.18,COKE
2257,2015-01-07,89.37,89.4900,88.0000,89.13,COKE
2258,2015-01-06,89.20,89.6300,88.0000,88.59,COKE
2259,2015-01-05,89.07,90.0273,88.3687,89.20,COKE


In [3]:
# 「Stock」でグループ化
gb = df.groupby('Stock')

gb.groups.keys()

dict_keys(['AAPL', 'COKE', 'TSLA'])

In [4]:
# アップル社のDataFrame
df_aapl = gb.get_group('AAPL')

# Traceを作成
trace = go.Candlestick(
    x=df_aapl['Date'], 
    open=df_aapl['Open'],
    close=df_aapl['Close'],
    high=df_aapl['High'],
    low=df_aapl['Low']
)   # アップル社株価のローソク足チャート

# 独自テンプレートを読み込み
with open('custom_white.json') as f:
    custom_white_dict = json.load(f)
    template = Template(custom_white_dict)

# Layoutを作成
layout = go.Layout(
    template=template,
    title='Hello-World-Stock dataset',
    xaxis={'title': 'Date'},
    yaxis={'title': 'Stock'}
)

# Figureを作成
figure = go.Figure(trace, layout)

figure

In [5]:
# 陽線の色のlist
inc_colors = [
    px.colors.sequential.Purp[2],
    px.colors.sequential.Redor[2],
    px.colors.sequential.Mint[2]
]

# 陰線の色のlist
dec_colors = [
    px.colors.sequential.Purp[6],
    px.colors.sequential.Redor[6],
    px.colors.sequential.Mint[6]
]

inc_colors

['rgb(209, 175, 232)', 'rgb(241, 156, 124)', 'rgb(137, 192, 182)']

In [6]:
# Traceのlistを作成
traces = []
for (group, df_group), inc_color, dec_color in zip(gb, inc_colors, dec_colors):
    trace = go.Candlestick(
        x = df_group['Date'], 
        open=df_group['Open'],
        close=df_group['Close'],
        high=df_group['High'],
        low=df_group['Low'],
        name=group,                         # Traceの名前
        increasing_line_color=inc_color,    # 陽線の色
        decreasing_line_color=dec_color,    # 陰線の色
    )   # 株価のローソク足チャート
    traces.append(trace)

# Figureを作成
figure = go.Figure(traces, layout)

figure

In [7]:
# コカ・コーラ社株価のDataFrame
df_coke = gb.get_group('COKE')

# Figureを作成
figure = make_subplots(rows=1, cols=2)

# Figureに散布図のTraceを追加
figure.add_trace(
    go.Scatter(x=df_aapl['Close'], y=df_coke['Close'], mode='markers', name='AAPL:COKE'),
    row=1,
    col=1
)

# Figureに折れ線グラフのTraceを追加
figure.add_trace(
    go.Scatter(x=df_aapl['Date'], y=df_aapl['Close'], mode='lines', name='AAPL close'),
    row=1,
    col=2
)
figure.add_trace(
    go.Scatter(x=df_coke['Date'], y=df_coke['Close'], mode='lines', name='COKE close'),
    row=1,
    col=2
)

# FigureのLayoutを更新
figure.update_layout(
    template=template,
    title='Hello-World-Stock dataset',
    xaxis={'title': 'AAPL close'},
    yaxis={'title': 'COKE close'},
    xaxis2={'title': 'Date'},
    yaxis2={'title': 'Stock'}
)

figure

In [8]:
# FigureWidgetを作成
widget = go.FigureWidget(make_subplots(rows=1, cols=2))

# FigureWidgetに散布図のTraceを追加
widget.add_trace(
    go.Scatter(x=df_aapl['Close'], y=df_coke['Close'], mode='markers', name='AAPL:COKE'),
    row=1,
    col=1
)

# FigureWidgetに折れ線グラフのTraceを追加
widget.add_trace(
    go.Scatter(x=df_aapl['Date'], y=df_aapl['Close'], mode='lines+markers', name='AAPL close'),
    row=1,
    col=2
)
widget.add_trace(
    go.Scatter(x=df_coke['Date'], y=df_coke['Close'], mode='lines+markers', name='COKE close'),
    row=1,
    col=2
)

# マーカーの標準サイズの配列
N = len(df_aapl)
default_scatter_size = 6
default_sizes = np.stack([
    np.full(N, fill_value=default_scatter_size),
    np.full(N, fill_value=0),
    np.full(N, fill_value=0),
], axis=0)


def hover_function(trace:go.Trace, points:callbacks.Points, selector:callbacks.InputDeviceState) -> None:
    """マーカーがフォーカスされた際のコールバック関数

    Args:
        trace (go.Trace): 対称のTrace
        points (callbacks.Points): フォーカスされたポイント
        selector (callbacks.InputDeviceState): セレクター
    """
    focused_size = 16   # フォーカスされたポイントのマーカーサイズ

    # 変更するマーカーサイズの配列
    size_array = default_sizes.copy()   # 標準サイズの配列からコピー
    if points.point_inds != []:
        index = points.point_inds[0]            # 散布図でフォーカスされたポイントのインデックス
        size_array[:, index] = focused_size     # 散布図と折れ線グラフの両方を大きくする
    
    # マーカーの更新
    with widget.batch_update():
        traces = widget.data
        for trace, size in zip(traces, size_array):
            trace.marker.size = size

def unhover_function(trace:go.Trace, points:callbacks.Points, selector:callbacks.InputDeviceState) -> None:
    """フォーカス解除された際のコールバック関数

    Args:
        trace (go.Trace): 対称のTrace
        points (callbacks.Points): フォーカス解除されたポイント
        selector (callbacks.InputDeviceState): セレクター
    """
    # 標準サイズのマーカーサイズに更新
    size_array = default_sizes.copy()
    with widget.batch_update():
        traces = widget.data
        for trace, size in zip(traces, size_array):
            trace.marker.size = size

In [9]:
# FigureWidgetのLayoutを更新
widget.update_layout(
    template=template,
    title='Hello-World-Stock dataset',
    xaxis={'title': 'AAPL close'},
    yaxis={'title': 'COKE close'},
    xaxis2={'title': 'Date'},
    yaxis2={'title': 'Stock'},
)

traces = widget.data
traces[0].on_hover(hover_function)
traces[0].on_unhover(unhover_function)

unhover_function(traces[0], plotly.callbacks.Points(), None)

widget

FigureWidget({
    'data': [{'marker': {'size': array([6, 6, 6, ..., 6, 6, 6])},
              'mode': 'markers',
              'name': 'AAPL:COKE',
              'type': 'scatter',
              'uid': '2aaa2f8a-4205-4d18-9dc4-c800c8e56b53',
              'x': array([169.23, 171.08, 170.6 , ..., 106.26, 106.25, 109.33]),
              'xaxis': 'x',
              'y': array([215.26, 216.83, 219.52, ...,  88.59,  89.2 ,  89.87]),
              'yaxis': 'y'},
             {'marker': {'size': array([0, 0, 0, ..., 0, 0, 0])},
              'mode': 'lines+markers',
              'name': 'AAPL close',
              'type': 'scatter',
              'uid': '89a6b32f-f7bb-4076-ac38-435bb5422e78',
              'x': array([datetime.datetime(2017, 12, 29, 0, 0),
                          datetime.datetime(2017, 12, 28, 0, 0),
                          datetime.datetime(2017, 12, 27, 0, 0), ...,
                          datetime.datetime(2015, 1, 6, 0, 0),
                          datetime.datet